In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
#%load_ext line_profiler

import tensorflow as tf
slim = tf.contrib.slim
tf.logging.set_verbosity(tf.logging.INFO)
sess_config = tf.ConfigProto()
#config.gpu_options.allow_growth=True
#config.gpu_options.per_process_gpu_memory_fraction=0.45

import sys
import os

ADE20K_DATA = '/gpfs01/bethge/data/ADE20K_2016_07_26'
COCO_DATA = '/gpfs01/bethge/share/mscoco/COCO'
PASCAL_VOC_DATA = '/gpfs01/bethge/data/PascalVOC'
MASK_RCNN_MODEL_PATH = 'Mask_RCNN/'
SLIM_MODELS_PATH = 'slim/'
TRANSFORMER_MODELS_PATH = 'transformer/'
SIAMESE_MASK_RCNN_PATH = '/gpfs01/bethge/home/cmichaelis/projects/2018-03_Siamese_Mask_RCNN/siamese-mask-rcnn/'

if MASK_RCNN_MODEL_PATH not in sys.path:
    sys.path.append(MASK_RCNN_MODEL_PATH)
if SIAMESE_MASK_RCNN_PATH not in sys.path:
    sys.path.append(SIAMESE_MASK_RCNN_PATH)
    
from samples.coco import coco
from samples.ade20k import ade20k
from mrcnn import utils
from mrcnn import model as modellib
from mrcnn import visualize
    
if SLIM_MODELS_PATH not in sys.path:
    sys.path.append(SLIM_MODELS_PATH)
if TRANSFORMER_MODELS_PATH not in sys.path:
    sys.path.append(TRANSFORMER_MODELS_PATH)
    
import utils as siamese_utils
import model as siamese_model
import config as siamese_config
import siamese_mrcnn_models as model_zoo
    
import time
import random
import numpy as np
import skimage.io
import imgaug
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (12.0, 6.0)

# from spatial_transformer import transformer

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:98% !important; }</style>"))

# Root directory of the project
ROOT_DIR = os.getcwd()

# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")

# Local path to trained weights file
COCO_MODEL_PATH = os.path.join(ROOT_DIR, "mask_rcnn_coco.h5")

In [ ]:
nopascal_classes = [8,10,11,12,13,14,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,60,62,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80]

In [ ]:
pascal_classes = np.array(range(1,81))[np.array([i not in nopascal_classes for i in range(1,81)])]

In [ ]:
# Load COCO/val dataset
val_dataset = siamese_utils.IndexedCocoDataset()
val_dataset.load_coco(COCO_DATA, "val", year=2017)
val_dataset.prepare()
val_dataset.build_indices()

In [ ]:
class TrainConfig(siamese_config.Config):
    # Set batch size to 1 since we'll be running inference on
    # one image at a time. Batch size = GPU_COUNT * IMAGES_PER_GPU
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1
    NUM_CLASSES = 1 + 1
    NAME = 'coco'
    EXPERIMENT = 'evaluation'
    # Reduced image sizes
    TARGET_MAX_DIM = 96
    TARGET_MIN_DIM = 75
    IMAGE_MIN_DIM = 400
    IMAGE_MAX_DIM = 512
    # Reduce model size
    FPN_CLASSIF_FC_LAYERS_SIZE = 512
    FPN_FEATUREMAPS = 256
    # Reduce number of rois at all stages
    RPN_ANCHOR_STRIDE = 1
    RPN_TRAIN_ANCHORS_PER_IMAGE = 64
    POST_NMS_ROIS_TRAINING = 500
    POST_NMS_ROIS_INFERENCE = 500
    TRAIN_ROIS_PER_IMAGE = 100
    DETECTION_MAX_INSTANCES = 30
    MAX_GT_INSTANCES = 30
    # Adapt NMS Threshold
    DETECTION_NMS_THRESHOLD = 0.5
    # Adapt loss weights
    LOSS_WEIGHTS = {'rpn_class_loss': 2.0, 
                    'rpn_bbox_loss': 0.1, 
                    'mrcnn_class_loss': 2.0, 
                    'mrcnn_bbox_loss': 0.5, 
                    'mrcnn_mask_loss': 1.0}
    TRAIN_BN = False
    
config = TrainConfig()
config.display()

### Visualization

In [ ]:
# Steps 100, 200, 300, ...
models = ['/gpfs01/bethge/home/cmichaelis/projects/2018-03_Siamese_Mask_RCNN/experiments/logs/coco_long/siamese_mrcnn_0310.h5']

for i in range(len(models)):
    # Create model object in inference mode.
    model = siamese_model.SiameseMaskRCNN(mode="inference", model_dir=MODEL_DIR, config=config)
    
    # set layers trainable for resnet weight loading
    epoch_index = int(models[i][-7:-3])
    if epoch_index <= 80:
        model.set_trainable("(mrcnn\_.*)|(rpn\_.*)|(fpn\_.*)")
    elif epoch_index <= 240:
        model.set_trainable("(res4.*)|(bn4.*)|(res5.*)|(bn5.*)|(mrcnn\_.*)|(rpn\_.*)|(fpn\_.*)")          
    else:
        model.set_trainable(".*")
    
    # Load weights
    model.load_weights(models[i], by_name=True)

In [ ]:
targets = []
images = []
rois = []
masks = []
class_ids = []
scores = []

In [ ]:
IMAGE_DIR = '/gpfs01/bethge/home/cmichaelis/datasets/COCO/val2017/'

# Load a random image from the images folder
# file_names = next(os.walk(IMAGE_DIR))[2]
# image = skimage.io.imread(os.path.join(IMAGE_DIR, random.choice(file_names)))
coco_image_ids = [val_dataset.image_info[id]["id"] for id in val_dataset.image_ids]

p = 1
while p < 2:

    # Load target
    category = 3
    random_image_id = np.random.choice(val_dataset.category_image_index[category])
    # random_image_id = 4005
    image = val_dataset.load_image(random_image_id)

    if np.abs(image.shape[0] / image.shape[1] - (3 / 2)) > 0.01:
        continue

#     target_image, _, target_class_ids, target_boxes, _ = \
#         modellib.load_image_gt(val_dataset, config, random_image_id, augmentation=False,
#                       use_mini_mask=config.USE_MINI_MASK)

#     box_ind = np.random.choice(np.where(target_class_ids == category)[0])   
#     tb = target_boxes[box_ind,:]
#     target = target_image[tb[0]:tb[2],tb[1]:tb[3],:]
    target1, window, scale, padding1, crop = siamese_utils.get_one_target(category, val_dataset, config, return_all=True)

    # Run detection
    results1 = model.detect([target1], [image], verbose=1)
    r1 = results1[0]
    cropped_target1 = siamese_utils.crop_target(target1, padding1)
    rois1 = r1['rois']
    masks1 = r1['masks']
    class_ids1 = r1['class_ids']
    scores1 = r1['scores']
    
    target2, window, scale, padding2, crop = siamese_utils.get_one_target(category, val_dataset, config, return_all=True)

    # Run detection
    results2 = model.detect([target2], [image], verbose=1)
    r2 = results2[0]
    cropped_target2 = siamese_utils.crop_target(target2, padding2)
    rois2 = r2['rois']
    masks2 = r2['masks']
    class_ids2 = r2['class_ids']
    scores2 = r2['scores']

#     cropped_target = siamese_utils.crop_target(target, padding)
#     siamese_utils.display_siamese_instances2(cropped_target, image, r['rois'], r['masks'], r['class_ids'], r['scores'])

#     targets.append(cropped_target)
#     images.append(image)
#     rois.append(r['rois'])
#     masks.append(r['masks'])
#     class_ids.append(r['class_ids'])
#     scores.append(r['scores'])
    
    p = p + 1

In [ ]:
siamese_utils.display_siamese_instances_teaser([cropped_target1, cropped_target2],
                                               [image]*2,
                                               [rois1, rois2],
                                               [masks1, masks2],
                                               [class_ids1, class_ids2],
                                               [scores1, scores2],
                                               figsize=(8 + 2 + 8 + 2 + 8, 12), fontsize=30)

In [ ]:
targets = []
images = []
rois = []
masks = []
class_ids = []
scores = []

p = 0
while p < 16:
    # Load target
    category = np.random.choice(list(range(1,81)))
    random_image_id = np.random.choice(val_dataset.category_image_index[category])
    # random_image_id = 4005
    image = val_dataset.load_image(random_image_id)

    if np.abs(image.shape[0] / image.shape[1] - (2 / 3)) > 0.1:
        continue
    
    target, window, scale, padding, crop = siamese_utils.get_one_target(category, val_dataset, config, return_all=True, target_size_limit=20)

    # Run detection
    results = model.detect([target], [image], verbose=0)
    r = results[0]
    cropped_target = siamese_utils.crop_target(target, padding)
    
    targets.append(cropped_target)
    images.append(image)
    rois.append(r['rois'])
    masks.append(r['masks'])
    class_ids.append(r['class_ids'])
    scores.append(r['scores'])
    
    p = p+1
    
siamese_utils.display_siamese_instances2(targets, images, rois, masks, class_ids, scores, figsize=(16, 16 * 2 / 3),
                                         target_shift=2)

In [ ]:
siamese_utils.display_siamese_instances2(targets, images, rois, masks, class_ids, scores, figsize=(16, 16 * 2 / 3),
                                         target_shift=2)

In [ ]:
siamese_utils.display_siamese_instances2(targets, images, rois, masks, class_ids, scores, figsize=(16, 16 * 2 / 3),
                                         target_shift=10)

In [ ]:
type(target)

In [ ]:
4872, 4005

In [ ]:
random_image_id

In [ ]:
siamese_utils.display_siamese_instances2(target[:, 28:-28,:], image, r['rois'], r['masks'], r['class_ids'], r['scores'])

In [ ]:
target.shape

In [ ]:
padding

In [ ]:
target[slice(21, 96 - 21), slice(0, 96),:]

In [ ]:
plt.imshow(target[slice(21, 96 - 21), slice(0, 96),:])